In [33]:
import requests
import threading
import random
import logging
import json
import time
from queue import Queue
from bs4 import BeautifulSoup

In [60]:
def run_time(func):
    def wrapper(*args, **kw):
        start = time.time()
        func(*args, **kw)
        end = time.time()
        print('running', end-start, 's')
    return wrapper

In [2]:
class UserAgent(object):
    __instance_lock = threading.Lock()
    __init_flag = False
    
    def __init__(self):
        if self.__init_flag is False:
            print('init UserAgent')
            self.__agents_pool = list()
            with open('./UserAgents/useragents.txt','r') as read_ob:
                for line in read_ob.readlines():
                    self.__agents_pool.append(line.strip())
            self.__init_flag = True
    
    def __new__(cls, *args, **kwargs):
        if not hasattr(UserAgent, "_instance"):
            print('new UserAgent')
            with UserAgent.__instance_lock:
                if not hasattr(UserAgent, "_instance"): 
                    UserAgent._instance = object.__new__(cls)
        return UserAgent._instance
    
    def get_useragent_randomly(self):
        return random.choice(self.__agents_pool)
    

In [128]:
class Crawl():

    def __init__(self):
        self._session = None
        self._headers = {
            'User-Agent':UserAgent().get_useragent_randomly(),
            'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language':'zh-cn',
            'Connection': 'keep-alive',
            'Accept-Encoding': 'gzip, deflate',
        }
        self._proxies = [
            {"http":"112.111.217.114:9999"},
            {"http":"180.118.128.118:9000"},
            {"http":"171.11.29.217:9999"},
            {"http":"120.83.109.191:9999"}
        ]
        
    def request_get(self, url, **kwargs):
        logging.info('scraping {}...'.format(url))
        try:
            response = requests.get(url, headers=self._headers, proxies=random.choice(self._proxies), **kwargs)
            if response.status_code == 200:
                return response
            logging.error('get invalid status code %s while scraping %s', response.status_code, url)
        except requests.RequestException:
            logging.error('error occurred while scraping %s', url, exc_info=True)
        else:
            logging.info('scraping {} finished'.format(url))
     
    def request_post(self, url, **kwargs):
        logging.info('scraping {}...'.format(url))
        try:
            response = requests.post(url, headers=self._headers, proxies=random.choice(self._proxies), **kwargs)
            if response.status_code == 200:
                return response
            logging.error('get invalid status code %s while scraping %s', response.status_code, url)
        except requests.RequestException:
            logging.error('error occurred while scraping %s', url, exc_info=True)
        else:
            logging.info('scraping {} finished'.format(url))
    
    def session_get(self, url, **kwargs):
        self.check_session()
        logging.info('scraping {}...'.format(url))
        try:
            response = self._session.get(url, headers=self._headers, proxies=random.choice(self._proxies), **kwargs)
            if response.status_code == 200:
                return response
            logging.error('get invalid status code %s while scraping %s', response.status_code, url)
        except requests.RequestException:
            logging.error('error occurred while scraping %s', url, exc_info=True)
        else:
            logging.info('scraping {} finished'.format(url))
    
    def session_post(self, url, **kwargs):
        self.check_session()
        logging.info('scraping {}...'.format(url))
        try:
            response = self._session.post(url, headers=self._headers, proxies=random.choice(self._proxies), **kwargs)
            if response.status_code == 200:
                return response
            logging.error('get invalid status code %s while scraping %s', response.status_code, url)
        except requests.RequestException:
            logging.error('error occurred while scraping %s', url, exc_info=True)
        else:
            logging.info('scraping {} finished'.format(url)) 

    def get_session(self):
        return self._session
    
    def set_session(self, session):
        self._session = session
        return True
    
    def save_cookies(self, user):
        cookieJar = requests.cookies.RequestsCookieJar()
        for cookie in user.cookies:
            cookieJar.set(cookie.name,cookie.value)
        for cookie in user.headers['Set-Cookie'].split(";"):
            key = cookie.split('=')[0]
            value = cookie.split('=')[1]
            cookieJar.set(key,value)
        return cookieJar
    
    def check_session(self):
        if self._session == None:
            self._session = requests.session()
        return
    
    def add_header(self, headers):
        for key, value in headers.items():
            self._headers[key] = value


In [129]:
class DoubanUser(Crawl):
    __instance_lock = threading.Lock()
    __init_flag = False
       
    def __init__(self):
        if self.__init_flag is False:
            super(DoubanUser, self).__init__()
            self.__user = None
    #         self._cookies = None
            self.__login()
            self.__init_flag = True

    def __new__(cls, *args, **kwargs):
        if not hasattr(DoubanUser, "_instance"):
            print('new Douban')
            with DoubanUser.__instance_lock:
                if not hasattr(Douban, "_instance"): 
                    DoubanUser._instance = object.__new__(cls)
        return DoubanUser._instance
    
    def __login(self):
        post_data = {
            'name':'18664678368',
            'password':'Ljc970412',
            'remember':'false'
        }
        self.add_header({
            "Referer":'https://accounts.douban.com/passport/login'
        })
        user = self.session_post('https://accounts.douban.com/j/mobile/login/basic', data=post_data)
        login_detail = json.loads(user.text)
        if login_detail['status'] == 'success':
            print('login success!')
#             self.__cookies = self.save_cookies(user)
        else:
            print('login failed!')
        self.__user =  user.text
    
    def get_user_info(self):
        return self.__user

In [130]:
class Douban(Crawl):
    
    def __init__(self, douban_user):
        super(Douban, self).__init__()
        self.set_session(douban_user.get_session())
    
    def search(self, query, cat=''):
        res = dict()
        params = {
            'q':query,
            'cat':cat
        }
        count=0
        response = self.session_get('https://www.douban.com/search', params=params)
        soup = BeautifulSoup(response.content, 'html.parser')
        results = soup.find(class_='result-list').find_all(class_='result')
        for result in results:
            pic = result.find(class_='pic')
            content = result.find(class_='content')
            img = pic.img.get('src')
            link = content.a.get('href')
            name = content.a.text
            description = content.p.text if content.p else ''
            res[count]={
                'name':name,
                'img':img,
                'link':link,
                'description':description
            }
            count+=1
        return res

In [131]:
class DoubanMovie(Douban):
    
    @run_time
    def get_comments(self, **kwargs):
        
        count = 0
        queue = Queue()
        producer_running = True
        
        def producer(url=None, movie_id=None, page_max=50, page=0):
            nonlocal producer_running
            if producer_running is True and page <= page_max:
                movie_url = None
                if movie_id is not None:
                    movie_url = 'https://movie.douban.com/subject/{}/comments'.format(str(movie_id))
                else:
                    movie_url = url
                print("开始爬取第{0}页***********************************************************************：".format(page+1))
                params={
                    'start':page*20,
                    'limit':20,
                    'status':'P',
                    'sort':'new_score'
                }
                html = self.session_get(movie_url, params=params)
                soup = BeautifulSoup(html.content, 'html.parser')
                comments = soup.find(id='comments').find_all(class_='comment-item')
                if len(comments) > 1:
                    queue.put(comments)
                    producer(url=movie_url, page=page+1, page_max=page_max)
                else:
                    producer_running=False
            else:
                producer_running=False
            
        def customer():
            nonlocal count
            while producer_running is True or queue.empty() is False:
                comments = queue.get()
                for comment in comments:
                    content = comment.find(class_='comment-content').text
                    user_name = comment.find(class_='comment-info').a.text
                    comment_time = comment.find(class_='comment-info').find(class_='comment-time').attrs['title']
                    print('comment:{}, user_name:{}, comment_time:{}'.format(content,user_name,comment_time))
                    count += 1
                print("count={}".format(count))
                time.sleep(int(random.choice([0.5, 0.2, 0.3])))
                
        threads = list()
        
        thread_p = threading.Thread(target=producer, kwargs=kwargs)
        thread_p.start()
        threads.append(thread_p)
        
        for _ in range(10):
            thread_c = threading.Thread(target=customer)
            thread_c.start()
            threads.append(thread_c)
            
        for thread in threads:
            thread.join(timeout=0.5)

In [75]:
class DoubanMovie(Douban):
    
    @run_time
    def get_comment(self, url=None, movie_id=None, page_max=50):
        movie_url = None
#         if movie_name is not None:
            
        if movie_id is not None:
            movie_url = 'https://movie.douban.com/subject/{}/comments'.format(str(movie_id))
        else:
            movie_url = url
        count = 0
        for page in range(page_max+1):
            print("开始爬取第{0}页***********************************************************************：".format(page+1))
            params={
                'start':page*20,
                'limit':20,
                'status':'P',
                'sort':'new_score'
            }
            html = self.session_get(movie_url, params=params)
            print(html.url)
            soup = BeautifulSoup(html.content, 'html.parser')
            comments = soup.find(id='comments').find_all(class_='comment-item')
            if len(comments) > 1:
                for comment in comments:
                    content = comment.find(class_='comment-content').text
                    user_name = comment.find(class_='comment-info').a.text
                    comment_time = comment.find(class_='comment-info').find(class_='comment-time').attrs['title']
                    print('comment:{}, user_name:{}, comment_time:{}'.format(content,user_name,comment_time))
                    count += 1
                print("count={}".format(count))
                time.sleep(int(random.choice([0.5, 0.2, 0.3])))
            else:
                print("大约共{0}页评论".format(page+1))
                break
            
#     def get_review_by_url(self, url , page_max=50):
#         res = dict()
#         self.set_url(url)
#         count = 0
#         for page in range(page_max+1):
#             print("开始爬取第{0}页***********************************************************************：".format(page+1))
#             self.set_params({
#                 'start':page*20
#             })
#             html = self.session_get()
#             soup = BeautifulSoup(html.content, 'html.parser')
#             reviews = soup.find(class_='review-list').find_all(class_='review-item')
#             for review in reviews:
#                 user = review.find(class_='main-hd')
#                 user_name = user.find(class_='name').text
#                 user_link = user.find(class_='name').a.get(href)
                
#                 content = review.find(class_='main-bd')
#                 title = content.find('h2').text
#                 link = content.find('h2').a.get('href')
#                 self.set_url(link)
#                 review_response = self.request_get()
#                 review_soup = BeautifulSoup(review_response, 'html.parser')
#                 article = review_soup.find(class_='article')
#                 review_content = article.find(class_='review-content').text
#                 res[title] = {
#                     'user_name':user_name,
#                     'user_link':user_link,
#                     'link':link,
#                     'review_content':review_content
#                 }
#             return res
        
#     def get_review_by_id(self, movie_id, page_max=50):
#         url = 'https://movie.douban.com/subject/{}/reviews'.format(str(movie_id))
#         self.get_review_by_url(url, page_max)
    
    def search(self, movie_name):
        result = super().search(movie_name, cat='1002')
        return result

In [132]:
user = DoubanUser()

new Douban
login success!


In [133]:
user.get_user_info()

'{"status":"success","message":"success","description":"处理成功","payload":{"account_info":{"name":"Ljc970412","weixin_binded":false,"phone":"18664678368","avatar":{"medium":"https://img3.doubanio.com\\/icon\\/user_large.jpg","median":"https://img1.doubanio.com\\/icon\\/user_normal.jpg","large":"https://img3.doubanio.com\\/icon\\/user_large.jpg","raw":"https://img3.doubanio.com\\/icon\\/user_large.jpg","small":"https://img1.doubanio.com\\/icon\\/user_normal.jpg","icon":"https://img3.doubanio.com\\/pics\\/icon\\/user_icon.jpg"},"id":"217184556","uid":"217184556"}}}'

In [134]:
dm_crawl = DoubanMovie(user)

In [135]:
dm_crawl.get_comments(url='https://movie.douban.com/subject/30171424/comments')

开始爬取第1页***********************************************************************：
开始爬取第2页***********************************************************************：comment:
还是推荐的，大场面和枪战音效的影院效果不错，但不吐不快：角色动机和转变都极为生硬，说白了故事不是剧情推进，而是导演推进，全员工具人，倪妮角色（谎言）戏份应该是刘青云来完成，这才有因果救赎的宿命感，如今刘青云存在感太低了。邱礼涛量级最大的行活儿，手快精良，不一定好但必须有自我表达：“社会学家”又拿出了当年拍《的士判官》那股劲头，时代不同了，他不再不挥舞屠刀改玩炸弹。PS，刘德华太帅了！第一集自爆的小哥这次又爆了
, user_name:乌鸦火堂, comment_time:2020-12-24 11:49:39

comment:
在当下这个语境里，敢把整个香港炸翻的也只有邱礼涛了。同样是香港导演，同样塑造英雄，邱礼涛和林超贤简直形成了两种极端，一个奏挽歌一个奏颂歌。《拆弹专家2》拍的是被集体反复利用后抛弃的个体，是制度的弃子，充满悲情，《紧急救援》拍的是大无畏的英雄主义，是为了人民安全可以不顾家人和自己、一股脑冲锋陷阵的救人机器。邱礼涛还是香港电影的打工人，仍然能对个体命运投注悲悯和同情，而林超贤已经沉迷在主旋律叙事当中了，只能用“国旗下的讲话”来包装出一个个教科书式的标兵。
, user_name:一口大井子, comment_time:2020-12-24 03:06:13
comment:
拆弹专家的救赎，身残志坚功夫好，十个特警全撂倒。开头机场爆炸音效细腻，层次丰富，槽点不少，硬伤很多，整体一般，中规中矩。倪妮华仔的感情戏，华仔的幡然醒悟等等，都只是为节奏服务的功能性转折。谢君豪说天才不应该被社会打败，在南海十三郎里被打败了一次，在拆弹专家里又被打败一次，好惨。德华买的诺基亚6120c是我高中时代梦寐以求的机型啊～
, user_name:阿孝咕, comment_time:2020-12-23 23:38:47
comment:
这哪里是拆弹2这是无间道4……各路反转顺滑到不行。BUG这么少的片子近期很难得了。除了有明显反社会

开始爬取第4页***********************************************************************：comment:
延续了香港电影中“后九七”症候群的疾病意象：断腿→失忆→旧身份失效→接入假肢→重新植入记忆→新身份确立，这样的叙事编码是对香港人身份焦虑、自我认同危机的一贯指涉（《我是谁》《紫雨风暴》《半支烟》《黄飞鸿之西域雄狮》）；同时也有对港产时装动作片中个人与体制二元对立、选择困难相关表征的承继：用完即弃→我不是疯是痛→你信我就是警察不信我就是恐怖分子→把这个烂制度变成灰烬，主体人物形象的塑造一直朝着质疑、愤怒、癫狂、自毁的路径在发展（《无间道》《两个只能活一个》《伊波拉病毒》《扫毒2》）。邱礼涛的厉害之处是能在三级片、B级片、恐怖片衰亡之后重新找到一条表达通道，而且竟在众目睽睽之下于作品中大量投射与当前时局有关的敏感内容。只要还有这样的导演在，真的不能说香港电影已死。
, user_name:孟浪, comment_time:2020-12-25 16:26:51

comment:
看完《拆弹专家2》点映
剧情老套，逻辑冲突。情绪靠BGM、表达基本靠吼、反派智商低到对着反贼说计划而不起疑心。还要抄一波《V字仇杀队》的地铁爆炸案
（PS：HK的治安真的被黑得好惨啊）
, user_name:V for Vendetta, comment_time:2020-12-24 01:01:00
comment:
首映 全程靠吼 刘德华吼完 刘青云吼 刘德华继续吼  tututu+mv+tutututututu 一部大家都有病 的电影 
, user_name:Tocca, comment_time:2020-12-17 20:37:04
comment:
牛逼啊牛逼！“置之死地而幸存，陷之绝境而复生”。30年过去了，华语电影终于出现了一部超越林岭东的《圣战风云》的反恐片（which means超越了诺兰以及全世界除了《盗日者》之外的同题材电影）。正看是个好人误入歧途找回天良大破乱党的故事。反看则是忠良受辱逼上梁山最后看在恋人面子上放这世界一马的反社会传奇。白衣白头的谢君豪与黑衣华仔对撞坠海的一刹那，“兄弟情”是真是假已不重要，冲天的水柱是一腔怒火对不公制度的怒吼，完爆那啥《黑暗骑士崛起》。“失忆者颠覆自己策划

开始爬取第7页***********************************************************************：comment:
太燃！年度必看港片！无论从叙事还是场面都是对第一集的全面超越。没想到邱礼涛竟然还藏有如此大的潜能，实在太稳。爆炸追逐层层递进，在“谍影重重”和“反恐24小时”间无缝切换，既是身份迷局，也是爆炸大片。每一次回述，刘德华都经历一次身份的错位，善与恶都潜藏在这个角色的内心之中。天使与魔鬼，争夺一个灵魂，是最贴切的潜在文本。
, user_name:西帕克, comment_time:2020-12-24 17:29:22

comment:
邱礼涛从香港的约翰·卡朋特变成了托尼·斯科特。
, user_name:皮革业, comment_time:2020-12-26 12:27:33
comment:
#電影院# 倪妮的正確打開方式！差不多跟《寒戰2》里的文咏珊一樣驚艷了，她的形象真的適合職業精英女性千萬不要再演傻白甜了。刘德华的气质用来扮演道貌岸然内心挣扎又逐渐黑化的腹黑角色好带感啊👍可惜的是刘青云没什么发挥空间未能物尽其用
, user_name:游牧人·芳汀, comment_time:2020-12-24 23:49:41
comment:
作为贺岁档，算是年末的惊喜，双刘组合一出现，就感觉港片的精髓还在，刘德华加刘青云的暗战组合一别二十年，又重现江湖，太让人上头了，完全重回港式警匪片的黄金时代的感觉。和第一部相比拆弹难度也是不断地升级。倪妮的特警扮相不错，场面很燃的大爽片，还意外收获泪点。
, user_name:张大水, comment_time:2020-12-24 15:59:04
comment:
就是面对这样一个病入膏肓的社会啊，可最终还是要靠弃子来救。用尽即弃的功利使然，变废为宝也是自私透顶的行径，而往往令人悲愤或绝望的，不是因为习以为常的恶，而是因为冠冕堂皇的正。邱礼涛真敢啊，不只是炸青马大桥或赤腊角机场的生猛，更是敢让正角（刘德华与潘乘风）走向崩坏的极端，这种反差把束手束脚的规矩破得痛快非凡。刘青云、姜皓文、谢君豪等等演员依然好得没话说，而邱礼涛、刘德华与倪妮终于超越自己。四星半。
, user_name:Mr. Infamous, comment_time:2020-1

running 5.542935848236084 s
开始爬取第10页***********************************************************************：comment:
之前看过首映的朋友说这部堪比好莱坞大片，所以每天都盯着它的上映时间，现在第一时间看完电影，果然感觉不负期待啊！用一句话总结刘德华在这部电影里的表演——对这部电影来说，刘德华已经不仅仅只是一个演员那么简单，某些瞬间，我感觉他就是潘乘风本人。
, user_name:金田一二三, comment_time:2020-12-24 09:56:05

comment:
双刘的角色换换多好，刘德华怎么演都不像反社会分子，刘青云倒是越来越阴鸷了。潘乘风浑身上下没有一点像残疾人，整部影片没有涉及半点真实残疾人的生活境况。庞玲和董吵架突然上升到“牺牲一个救一百个”（电车难题），体现了二极管时代的典型思维。“阿风我带你去看医生吧”“我看过了也吃了药”“我是指心理医生”，草，笑得我想吐。19年hk黑暴过去没多久坐在电影院里看这个，能强烈地感受到电影的时效性，不过我总感觉导演有点两面派，骂恐怖分子是为了赚大陆人民的感情💰，影射“comrades”=terrorists才是真……😅
, user_name:云理守, comment_time:2020-12-28 01:44:22
comment:
3.5吧，到我打分前这片儿分数已经上下浮动两回了，8到8.1再到8再到8.1，嗯这很符合我看这部片儿的体验，就是一口蛋糕一口蒜的感觉，前一场戏觉得哇好猛啊这爆炸这断腿死这老多警察，后一场就呃真这啥直男台词，前一刻哇拉横幅了真的好猛！后一秒就复生会随时为老板而死，哈哈这什么中二恐怖组织，最后炸桥我情绪都到那儿了，倪妮来一句“下辈子一定要记住我叫庞凌”，硬是给我整笑了，你是预定了下辈子的身份证还是怎么？整体文戏台词国语配音拉胯，没台词的戏都还不错。最后想说这戏里的人质太可怜了，香港警察根本就不care你们的死活好嘛！
, user_name:渡边, comment_time:2020-12-25 22:24:27
comment:
信我就是警察，不信我就是恐怖分子，年度院线华语电影最佳！好一个逼良为娼！作为一部类型片，该有的都有了，还夹带了不少私货，瞧把你给能的！扬眉吐

开始爬取第13页***********************************************************************：comment:
8.0/10【华语片年度最佳】
其实题眼就在当初潘乘风对庞玲说的气话里了：“信不信我造一颗炸弹，然后拆掉它，让全世界都肯定我的能力？”所以在文本上，本片已然做到了类型片的最高阶：立意两不得罪，无所谓谁吃下谁，却偏偏还能有个正向上扬的结局。
诚然，剧作手法上有些笨拙：以人物带故事，却没能用故事点人物。故而后半段两处逻辑链羸弱，马潘两人关系的隐线也没有做好。可当看到黑底白字的横幅竖起，又有谁能无动于衷呢？
这就是华语片中的《黑暗骑士》，是国产类型片饱受桎梏节节败退后，港导为我们留住的最后曙光。
, user_name:眼睛瞪得像铜铃, comment_time:2020-12-24 21:39:40
comment:
场面宏大，剧本也有新意，没有过于明显的硬伤，总体而言值回票价。
, user_name:Tom, comment_time:2020-12-29 11:54:07
comment:
“我给你取名杨过，是叫你知过改过”
, user_name:废话队长, comment_time:2020-12-17 18:08:55
comment:
刘老板一句“我们做拆弹的，哪一次不是用命拼的”，差点说出我的眼泪，自动把拆弹俩字换成电影了。
, user_name:windywinky, comment_time:2020-12-23 22:41:31
comment:
确实想过迫不得已或者卧底这种开展，没有想到刘德华的黑化就是黑化，这种处理确实非常厉害。最后的结局有一点意难平吧，总幻想是大团圆ending，但实际上让潘乘风在桥上死去就是最好的结局了。活下来反而尴尬，他身上背着的人命和他在那个时候拯救的人命是不能等量计算的，加上他的记忆根本就不完整，那他到底是英雄还是罪犯呢？与其让他带着残缺的记忆尴尬的活下去，不如让这个重新被建立是非观的他死去吧，至少他背向大海坠落的那一刻，成为了自己心中的英雄。
, user_name:云浪春山, comment_time:2020-12-26 02:29:47
comment:
影院看效果特别好，动作场面紧张激烈，但文戏的剧情恐怕经不起仔细推敲，不过在影院当

开始爬取第16页***********************************************************************：comment:
视觉效果不错，看得好紧张，香港恐怖袭击要真这么多曾经穿梭在港铁的我简直后怕。人设都比较单薄，心理转变过程突然，反派动机不明显。不过这样过电影主要也不是为了讲好一个故事。珍惜和平，向用生命拯救生命的英雄致敬。 at 荔枝广场幸福蓝海

, user_name:阿紫, comment_time:2020-12-27 17:52:22

comment:
近年来港片在保持港风剧情一贯优势的前提下，制作越来越精良，机场爆炸片段最让我记忆深刻，火光滔天飞机瞬间化为灰烬效果震撼。真兄弟不会没有争执，女性角色穿插其中很好的调节了紧张的氛围。四星推荐。
, user_name:张霓, comment_time:2020-12-24 15:30:46
comment:
表演略程式化，扣一分，其他构思还挺好的
, user_name:Johny, comment_time:2020-12-25 12:50:08
comment:
酣畅过瘾的爽片
, user_name:荒林, comment_time:2020-12-26 16:49:05
comment:
产生身份焦虑的是刘德华不是谢君豪，只此一点就说明邱礼涛已然决绝地和97世代的香港电影割席。10年来所有的香港商业电影凝望“昨日之辉煌”迟迟不肯撒手，最后在树大招风里摆出风度“让一切随风”。只有邱礼涛凝视着那部凄厉的十年，徒手从血色模糊里挖出愤怒的心肺给陆港共赏。你笑话白发谢君豪是饮血魔头，但是倪妮那句“你信我你就是英雄，不信你就是恐怖分子”才是题眼。整个香港的爆炸都是刘德华自己的左右互搏，但你真的自信倪妮的读心术一生一世不会反伤到自己一分一毫吗。
, user_name:空空儒也, comment_time:2020-12-24 18:49:01
comment:
觉得挺好，但是没什么感觉…可能是因为看的国语配音，也可能是因为onecool调色，总之不是no good，可能这就是low good吧……low good low good也挺好。我给倪妮加一星……🤭
, user_name:海樹, comment_time:2020-12-29 15:56:3

开始爬取第19页***********************************************************************：comment:
8.0/10【华语片年度最佳】
其实题眼就在当初潘乘风对庞玲说的气话里了：“信不信我造一颗炸弹，然后拆掉它，让全世界都肯定我的能力？”所以在文本上，本片已然做到了类型片的最高阶：立意两不得罪，无所谓谁吃下谁，却偏偏还能有个正向上扬的结局。
诚然，剧作手法上有些笨拙：以人物带故事，却没能用故事点人物。故而后半段两处逻辑链羸弱，马潘两人关系的隐线也没有做好。可当看到黑底白字的横幅竖起，又有谁能无动于衷呢？
这就是华语片中的《黑暗骑士》，是国产类型片饱受桎梏节节败退后，港导为我们留住的最后曙光。
, user_name:眼睛瞪得像铜铃, comment_time:2020-12-24 21:39:40

comment:
场面宏大，剧本也有新意，没有过于明显的硬伤，总体而言值回票价。
, user_name:Tom, comment_time:2020-12-29 11:54:07
comment:
“我给你取名杨过，是叫你知过改过”
, user_name:废话队长, comment_time:2020-12-17 18:08:55
comment:
刘老板一句“我们做拆弹的，哪一次不是用命拼的”，差点说出我的眼泪，自动把拆弹俩字换成电影了。
, user_name:windywinky, comment_time:2020-12-23 22:41:31
comment:
确实想过迫不得已或者卧底这种开展，没有想到刘德华的黑化就是黑化，这种处理确实非常厉害。最后的结局有一点意难平吧，总幻想是大团圆ending，但实际上让潘乘风在桥上死去就是最好的结局了。活下来反而尴尬，他身上背着的人命和他在那个时候拯救的人命是不能等量计算的，加上他的记忆根本就不完整，那他到底是英雄还是罪犯呢？与其让他带着残缺的记忆尴尬的活下去，不如让这个重新被建立是非观的他死去吧，至少他背向大海坠落的那一刻，成为了自己心中的英雄。
, user_name:云浪春山, comment_time:2020-12-26 02:29:47
comment:
影院看效果特别好，动作场面紧张激烈，但文戏的剧情恐怕经不起仔细推敲，不过在影院

开始爬取第22页***********************************************************************：comment:
视觉效果不错，看得好紧张，香港恐怖袭击要真这么多曾经穿梭在港铁的我简直后怕。人设都比较单薄，心理转变过程突然，反派动机不明显。不过这样过电影主要也不是为了讲好一个故事。珍惜和平，向用生命拯救生命的英雄致敬。 at 荔枝广场幸福蓝海

, user_name:阿紫, comment_time:2020-12-27 17:52:22

comment:
近年来港片在保持港风剧情一贯优势的前提下，制作越来越精良，机场爆炸片段最让我记忆深刻，火光滔天飞机瞬间化为灰烬效果震撼。真兄弟不会没有争执，女性角色穿插其中很好的调节了紧张的氛围。四星推荐。
, user_name:张霓, comment_time:2020-12-24 15:30:46
comment:
表演略程式化，扣一分，其他构思还挺好的
, user_name:Johny, comment_time:2020-12-25 12:50:08
comment:
酣畅过瘾的爽片
, user_name:荒林, comment_time:2020-12-26 16:49:05
comment:
产生身份焦虑的是刘德华不是谢君豪，只此一点就说明邱礼涛已然决绝地和97世代的香港电影割席。10年来所有的香港商业电影凝望“昨日之辉煌”迟迟不肯撒手，最后在树大招风里摆出风度“让一切随风”。只有邱礼涛凝视着那部凄厉的十年，徒手从血色模糊里挖出愤怒的心肺给陆港共赏。你笑话白发谢君豪是饮血魔头，但是倪妮那句“你信我你就是英雄，不信你就是恐怖分子”才是题眼。整个香港的爆炸都是刘德华自己的左右互搏，但你真的自信倪妮的读心术一生一世不会反伤到自己一分一毫吗。
, user_name:空空儒也, comment_time:2020-12-24 18:49:01
comment:
觉得挺好，但是没什么感觉…可能是因为看的国语配音，也可能是因为onecool调色，总之不是no good，可能这就是low good吧……low good low good也挺好。我给倪妮加一星……🤭
, user_name:海樹, comment_time:2020-12-29 15:56:3

开始爬取第25页***********************************************************************：comment:
刘青云饰演的董卓文是干什么用的。
, user_name:直泣优胜劣汰, comment_time:2020-12-26 11:19:38

comment:
港片文艺复兴！
“我已经回不了头了。”
其实，破后而立，如果时机对了，也未必不是一件好事。
最后，倪女士，请保持现在这个势头！
, user_name:迷宫里的小白猫, comment_time:2020-12-25 19:07:47
comment:
安迪刘化身杰森伯恩上演失忆戏码，暗戳戳还是香港人的身份焦虑，所以和爆炸场面相比，更可怕的是倪妮的谎言，打着为你好的旗号就可以颠倒黑白，要是早年的邱礼涛作品，潘乘风就会发展成散布伊波拉病毒的黄sir，如今的形势就只好对标水浒，落得个悲情下场，好在癫佬能想出炸桥的绝招，死也死得硬核些。
, user_name:超cute侠, comment_time:2020-12-25 23:50:05
comment:
特效还是可以的
, user_name:Adiós, comment_time:2020-12-28 00:50:54
comment:
本年度最佳华语院线电影！另外，作为1999年的高考考生，永远不会忘记的语文全国卷作文题：假如记忆可以移植。谢谢倪妮，你点题了。
, user_name:clarity, comment_time:2020-12-25 01:39:21
comment:
城市的愤怒与绝望，我一个隔岸观火者都被击中了，两年来的新闻画面一再重现，和银幕叠化，无法想象在地市民看这片时的心情……
, user_name:淹然, comment_time:2020-12-28 00:29:07
comment:
個體面對體制不公不得不採取激進的抗爭手段，這一點又反被體制利用來為自己服務，並致使個體在身體和精神上雙重殘缺（殘疾＋失憶），最終個體通過個人化的愛情找到了自己的身份和任務，並以捨棄自己生命的方式贖罪。影片再次借用臥底這一香港警匪片的常見母題，來進行對香港生存境况的隱喻性的政治敘事（實則十分直露）。影片傳達的價值觀在其信奉者看來顯得天然正義，豆瓣最高贊短評就邱禮濤和林超賢

In [107]:
dm_crawl.get_comments(url='https://movie.douban.com/subject/30171424/comments')

开始爬取第1页***********************************************************************：
开始爬取第2页***********************************************************************：comment:
在当下这个语境里，敢把整个香港炸翻的也只有邱礼涛了。同样是香港导演，同样塑造英雄，邱礼涛和林超贤简直形成了两种极端，一个奏挽歌一个奏颂歌。《拆弹专家2》拍的是被集体反复利用后抛弃的个体，是制度的弃子，充满悲情，《紧急救援》拍的是大无畏的英雄主义，是为了人民安全可以不顾家人和自己、一股脑冲锋陷阵的救人机器。邱礼涛还是香港电影的打工人，仍然能对个体命运投注悲悯和同情，而林超贤已经沉迷在主旋律叙事当中了，只能用“国旗下的讲话”来包装出一个个教科书式的标兵。
, user_name:一口大井子, comment_time:2020-12-24 03:06:13
comment:
是晁盖也是宋江，是警察也是匪徒，是英雄也是疯子，是解铃人也是系铃人。为何不认识她还是会心动，不记得他还是会心痛，为何要在用人即弃的世界苟且，在万夫莫开的关口坚守。为了拯救别人不惜性命，为了证明能力舍掉自我。救一城来渡己的是圣人，灭众生来渡己的是魔鬼，不过一念之间，却得用一生偿还。
, user_name:西楼尘, comment_time:2020-12-21 22:41:43
comment:
观感好于第一部，香港地标爆破场面更大更多，但最让人惊喜的是，邱礼涛在条条框框下依旧不屈服，他的愤怒、他的控诉、他的摧毁欲都还在，身体残障、行刑主题、无政府主义，其实更应该与《扫毒2》放在一起，再往前勾连《的士判官》和《夜叉》。“我不是疯，我是痛”，“是这个社会病了”，“用完即弃”，“把这个烂制度变成灰烬”，这个乱世，实在太适合邱礼涛来宣泄了，他在为香港呐喊。
, user_name:阿历克斯, comment_time:2020-12-17 12:09:10
comment:
超出预期太多了，活生生把我的尿意憋回去了两个小时的那种好。自以为《无双》之后港片黯淡，《拆弹专家2》又燃了，纯正港味，商业品控拿捏死，又或许是《扫毒2》的试水成功，邱礼涛开始打磨内地观众胃口，有了这一部剧

开始爬取第4页***********************************************************************：comment:
在这个病毒肆虐的贫瘠年代，是时候该为变态的邱礼涛加冕了。“炸裂”这个词简直是为《拆弹专家2》量身打造的，谁能想到，毁完红磡隧道、中环地铁之后，他居然还能交出更加震撼的大场面。在这份S级动作片想象力面前，林超贤被衬得像个只会写八股文的小学生。“我不是疯，是痛”，也只有他才能在一部商业大制作里，让刘德华秒变黄秋生，替全香港诉说心声。美中不足是反转来得过早，为青马大桥爆破让道，没把反社会进行到底，也没能往人性最幽暗处再探寻。但跟他在审查系统眼皮底下变的帽子戏法相比，这瑕疵显得微不足道。新世代的里程碑，最适合为2020年定调收尾的电影，预言式地给所有时代情绪把脉问诊，华仔最后仿佛向《伊波拉病毒》投去凝重的回眸。
, user_name:晚不安, comment_time:2020-12-20 17:10:00
comment:
确实比第一部更好，还是“行活导演”邱礼涛导演这几年给人的感觉，就是可以娴熟的驾驭类型技巧，他的片子满眼都是干净利落的技巧，可以随意用技巧控制电影的节奏和观众的情绪。到了这部电影里，个人对抗体制的不公这个主题带有极强的煽动性，就算中段的不断闪回让人物变成了工具人，只要技巧用得好，故事翻转的快感和哐哐哐动作戏的荷尔蒙依旧可以阻挡绝大多数观众的理智。像发小设计，全是红色的密码之类的细节设计，就是功能性大于一切的设计，不高明，但却又有效，这就是类型片的秘密。★★★☆
, user_name:亵渎电影, comment_time:2020-12-24 21:18:20
comment:
剧本牛逼了。被体制“用后即弃”人，在还有利用价值的情况下，通过更改记忆拿回来再用。潘乘风最终的归化不是“被骗”的结果（他已经知道事情真相），而是源自人性本善。这一点无疑让电影的思想深度打了折扣，多少有些创作者自身的一厢情愿和被迫无奈。但就整个文本对香港与新属国之间的关系做出的隐喻和阐释，相当直接又精准。当然，就各方面制作来看，可以更好。
, user_name:把噗, comment_time:2020-12-24 21:47:15
comment:
有人说爱情线矫情，结局多余。其实末世港片很多会以97的

开始爬取第7页***********************************************************************：comment:
太燃！年度必看港片！无论从叙事还是场面都是对第一集的全面超越。没想到邱礼涛竟然还藏有如此大的潜能，实在太稳。爆炸追逐层层递进，在“谍影重重”和“反恐24小时”间无缝切换，既是身份迷局，也是爆炸大片。每一次回述，刘德华都经历一次身份的错位，善与恶都潜藏在这个角色的内心之中。天使与魔鬼，争夺一个灵魂，是最贴切的潜在文本。
, user_name:西帕克, comment_time:2020-12-24 17:29:22
comment:
邱礼涛从香港的约翰·卡朋特变成了托尼·斯科特。
, user_name:皮革业, comment_time:2020-12-26 12:27:33
comment:
#電影院# 倪妮的正確打開方式！差不多跟《寒戰2》里的文咏珊一樣驚艷了，她的形象真的適合職業精英女性千萬不要再演傻白甜了。刘德华的气质用来扮演道貌岸然内心挣扎又逐渐黑化的腹黑角色好带感啊👍可惜的是刘青云没什么发挥空间未能物尽其用
, user_name:游牧人·芳汀, comment_time:2020-12-24 23:49:41
comment:
作为贺岁档，算是年末的惊喜，双刘组合一出现，就感觉港片的精髓还在，刘德华加刘青云的暗战组合一别二十年，又重现江湖，太让人上头了，完全重回港式警匪片的黄金时代的感觉。和第一部相比拆弹难度也是不断地升级。倪妮的特警扮相不错，场面很燃的大爽片，还意外收获泪点。
, user_name:张大水, comment_time:2020-12-24 15:59:04
comment:
就是面对这样一个病入膏肓的社会啊，可最终还是要靠弃子来救。用尽即弃的功利使然，变废为宝也是自私透顶的行径，而往往令人悲愤或绝望的，不是因为习以为常的恶，而是因为冠冕堂皇的正。邱礼涛真敢啊，不只是炸青马大桥或赤腊角机场的生猛，更是敢让正角（刘德华与潘乘风）走向崩坏的极端，这种反差把束手束脚的规矩破得痛快非凡。刘青云、姜皓文、谢君豪等等演员依然好得没话说，而邱礼涛、刘德华与倪妮终于超越自己。四星半。
, user_name:Mr. Infamous, comment_time:2020-12

开始爬取第10页***********************************************************************：comment:
节奏紧凑，场面十分震撼。虽然是第二部，但是是一个完全独立的故事，还有意想不到的反转。善恶只在一念之间，友情、爱情……还有职业的操守。最喜欢刘德华和刘青云的对手戏，过瘾。
, user_name:扰扰, comment_time:2020-12-24 10:08:00

comment:
果然不负期待，电影中的娱乐性和政治性融合的恰到好处，两个小时全程被吸引住，追逐、枪战、爆破无不精彩，一面是无政府主义的恐怖分子，一面是除暴安良的反恐精英，潘乘风的“左右”互搏既复杂又真实，董卓文、庞玲的“情与法”选择同样深刻，生动可信又呈现多样人性。。而被动的身份失焦和主动的清除记忆也正好对应，沿袭了邱礼涛导演一贯风格与态度。
, user_name:亲切的昊子, comment_time:2020-12-24 20:08:12
comment:
IMAX国语版。is it getting better or do you feel the same? 这个世界已不可救药，毁灭才是唯一的解药，优秀的反转是反转前令人信以为真而不是漏洞百出，最后没炸成，噢，非剧透，开头就透了，对不起，我是警察，给我一个机会，我现在想选择做好人，你信吗？反正我不信，
, user_name:風知道, comment_time:2020-12-25 20:18:19
comment:
近年接触到烂片的概率越来越小了，没想到啊没想到，豆瓣8.1，也能是部大烂片，给1分是因为没有0分可给。
, user_name:Ringo, comment_time:2020-12-25 21:35:22
comment:
万用三板斧已齐活：轰炸感的劲爆场面、反转快节奏、偏执狠角色。其实一直很佩服港片的一个点是，总能在紧张的故事情节中，用非常精炼而精致的笔墨将人物见缝插针塑造起来，轻轻几笔将人物立住，让你情不自禁对人物产生代入感。
, user_name:王小叶儿, comment_time:2020-12-17 23:56:53
comment:
《拆弹专家2》是一个行为艺术，必须在特定的艺术“界”的政治环境之下被理解，并产生意义。邱礼涛以香港电影

running 5.841109991073608 s
开始爬取第13页***********************************************************************：comment:
就正常水平发挥吧！精彩的都在动作场面，尴尬的都在文戏爱情。感觉编剧是把几个拆弹的故事想明白了，其他的人物就工具人往里塞吧，别讲逻辑、更别谈流畅。倪妮的戏真的是太难看了，哎，脚趾抠出一套大平层。说男人戏基情戏的，不接受。
, user_name:薇羅尼卡, comment_time:2020-12-28 00:26:38

comment:
一个精神分裂的故事，身份，立场，社会将人四分五裂。香港电影始终在自我身份认同中摸索。也看到隐喻，只有“失忆”这样的外界大创伤，才能让一个愤怒的人切断怒火回归人性，相应的，社会性群体性的问题和压抑积累下的愤怒，除非类似二战这样的集体创伤，才能一切重建。刘德华的用后即弃，刘青云的控诉记忆篡改，还有自倪妮口中说出，信我是警察，不信我是恐怖分子，幼时回忆里回答帮我为何也要受罚，都值得玩味。
, user_name:嘚嘚嘚嘚嘚, comment_time:2020-12-27 01:46:38
comment:
文戏动作戏都非常棒，视听也好。倪妮好漂亮。
, user_name:自由的幻影, comment_time:2020-12-27 22:06:11
comment:
节奏还行，但情感处理太粗糙了，就，都是一派理所当然的气氛。
, user_name:端舞, comment_time:2020-12-24 22:02:07
comment:
快，准，狠。
, user_name:aof, comment_time:2020-12-25 19:33:59
comment:
《伯恩的身份》式谜团，《无间道》般经历，《记忆碎片》的洗脑，刘德华两次至关重要的抉择，令人唏嘘。哪有什么天性纯良，全是命运安排。天使和魔鬼，永远都是一个硬币的正反面。假如生活欺骗了你，有的人成为了小丑，有的人成为了罗夏。更多的人，委曲求全中苟活了一生。人间明暗，人间明暗啊。邱礼涛差点把香港炸了，杜比影院差点把我炸了。仿佛看了一场后摇演出，耳朵现在还在打鸣。
, user_name:巴伐利亞酒神, comment_time:2020-12-

开始爬取第16页***********************************************************************：comment:
满屏CG特效看的有点晕，港警巅峰还是无间道啊
, user_name:半岛墨鱼干, comment_time:2020-12-27 14:14:18

comment:
刘德华从头到脚都是戏，好人坏人一瞬间眼神的变化绝了。倪妮真的很适合这种飒女角色，集美丽智慧力量一体的女性真的好逗人爱啊～
, user_name:我傻笑你随意。, comment_time:2020-12-26 11:08:49
comment:
有点过誉了。一个反社会的恐怖分子被前女友一通嘴炮就反水了？那他成立恐怖组织的意义何在？我是没感受到救赎，反而有种脱裤子放哪啥的感觉。几个有代号的反派小弟送的都既容易又儿戏了。
, user_name:栗先生是不是一, comment_time:2020-12-27 17:04:52
comment:
越到后面越有点挂不住，不过人性就真的是喜欢这种炸炸炸的电影，全靠音乐轰轰轰，行吧
, user_name:陈裸, comment_time:2020-12-28 16:01:30
comment:
新时代《无间道》。十八年了，香港的不变与变，太唏嘘。炸吧，把香港炸平，让一切随风。 
, user_name:萨库拉, comment_time:2020-12-28 22:47:57
comment:
不是我喜欢的类型，等人的间隙看的，非常值得票价，冷静冷酷又有爆发力，工业技术相当不错，叙事上没有一句废话，香港演员的敬业和专业程度超出了我的想象。
, user_name:低首尘埃, comment_time:2020-12-27 16:29:09
comment:
我，邱粉。
, user_name:次非, comment_time:2020-12-24 23:26:16
comment:
本年度看的最好的电影😭😭😭刘德华演的角色太惨了，刘德华这个好人型演员在多年的锻炼中真的越来越好了。真的不是只有天赋性演员才可以演好戏啊。好久没看到刘青云了，他和刘德华这个搭档我很喜欢，想到了暗战。倪妮是我觉得内地少有的演技美貌都很高的女演员。
, user_name:取个名字吧, comment_tim

开始爬取第19页***********************************************************************：comment:
我这个人比较肤浅，拆弹电影炸的过瘾就给高分！有老牌港片的硬气，演员/替身武戏很拼，没有一个拉胯，好评，角色设定上明显倪妮就为平衡男女性别比例，不如大胆一些这个角色也找一位男演员演(⁎⁍̴̛ᴗ⁍̴̛⁎)
, user_name:瑾朵朵, comment_time:2020-12-26 16:09:42

comment:
不怎么样吧，
, user_name:驻马店凯鲁亚克, comment_time:2020-12-29 08:45:59
comment:
真tm爽！光凭动作逻辑的设计就已经完全值回票价了，有些特写其实挺出戏的，但无奈给的量太足够了，开场第一场戏就直接核弹把机场给炸了，然后告诉你这片土地上最多的情绪就是愤怒！//剪辑上故意给观众营造一种信息超载的感觉，这点跟《信条》挺像的，不过这片的故事没那么复杂，所以完成得还挺好的。//配乐绝对抢戏，几乎铺满了整部电影，但这片用得挺有效的，在视觉信息量超载的情况下，听觉上也不放过。导演用最直接的方式把情绪硬塞给你。//故事上玩的是谍影重重式罗生门叙事，虽然只是学了个皮毛，但是整个故事的完整性还真挺不错的。//我很喜欢片子里面的一些细节，比如举着黑底白字站在发布会现场，再比如一直压着的情绪释放出两个字“反抗”。
, user_name:Luke, comment_time:2020-12-25 02:16:12
comment:
。。。。这片为啥能得8分以上啊。反派简单弱智的可怜。翻转太生硬，为啥突然就反社会了啊？为啥突然又变好了啊？为啥说像《无间道》？哪里有卧底了？华仔自己都不知道自己是个卧底！

, user_name:zidane2311, comment_time:2020-12-25 23:06:24
comment:
三星四星犹豫了一下，看在谢君豪那边复生会的反派塑造的很有魅力多加一颗星。我想看谢君豪那边反派故事啊！！！本来是冲着刘青云去的，可感觉他的角色可有可无，略可惜……我竟然还妄想看到暗战cp的对决时刻 哈哈。【P.S：求求编剧们看看观众的呼声！可以不凑男女cp的！要是写不好男女主感情戏可以完全放弃的，省得那么尬还拖慢电影节

开始爬取第22页***********************************************************************：comment:
上一集炸港铁这集炸了青马大桥和机场，燃炸场面还是挺多的，确实花钱了。60岁的刘德华做各种危险动作，20年前和他势均力敌的刘青云给他当配。40岁的吴卓羲和刘浩龙去当配角当小喽啰看老哥哥们玩命。新人都没有。谢君豪还是很帅。倪妮还是美的，只是可惜是个工具人。
, user_name:玉木大河, comment_time:2020-12-24 22:37:43

comment:
倪妮太美了 大美人！！！
, user_name:バカモノ, comment_time:2020-12-24 21:35:48
comment:
没办法，入不了戏，刘德华的故作坚韧，杯具卧底还是脱不开无间道的影子，很多设定和反转还是简单套路了点，那些爆炸特效也差了点，这在好莱坞也就是B级片的水平。这算是香港遭受核弹攻击的第一次银幕化呈现吧？
, user_name:声色画报, comment_time:2020-12-26 10:13:16
comment:
三星半。开场就非常炸裂，全程节奏紧凑，高潮部分也做得很好。作为类型片的动作枪战，是内地同类型的标杆，这充分说明敢称得上华语电影工业的，还得看香港。逻辑上肯定有很多bug，但胜在节奏好，所以并不会让人出戏。刘青云和刘德华《暗战》之后的合作火花差了不少，但华仔是真拼命，一时间以为成龙附体。导演虽然有话说，但他是邱礼涛，不是杜琪峰，所以几乎可以忽略不计。香港电影工业就如我之前所说，随风潜入夜，润物细无声，复生也可以很温柔。
, user_name:微光少女, comment_time:2020-12-26 20:46:04
comment:
剧情刺激反转很多，特效看的很真实，刘德华饰演的潘乘风和刘青云饰演的董卓文之间的对手戏很精彩，整片看起来有不少的大场面，呜呜呜呜呜呜也是很值得一看了
, user_name:江湖骗子, comment_time:2020-12-24 14:37:40
comment:
没有1好，拆弹专家就好好拆弹，老想讲大故事，恐怖分子弄的跟儿戏一样，这点应该学寒战
, user_name:走马, comment_time:2020-12-27 19:3

开始爬取第26页***********************************************************************：comment:
躲的了子弹，躲不了炸弹啊。
, user_name:私房歌, comment_time:2020-12-25 11:22:41
comment:
“信我就是警察，不信我就是恐怖分子！”邱礼涛真牛逼，将一足本的水浒传融进了香港新千年后的历史，并用潘乘风投诚后的际遇和姿态隐喻了香港未来几十年可能的遭遇。看邱礼涛的电影是充满间离感的，他意不在讲故事，而是在写论文。能巧妙把操纵记忆这样的攻壳式主题与香港近几年和未来几年可能的遭遇无缝融合，真是鬼才。说到大场面和动作场面的设计，感觉邱礼涛不去好莱坞拍大片真的太可惜了
, user_name:索尔之锤, comment_time:2020-12-26 17:30:27
comment:
年度最佳
, user_name:南泉, comment_time:2020-12-28 10:55:05
comment:
编剧还真是老套，好多对白直接得只为推动剧情服务，感情戏很烂，倪妮与华仔完全没有CP感，但并不妨碍这是部好看的动作戏。虽然刘德华演什么都是他自己，可确实还蛮拼的，要的就是这行云流水的过程。刘青云没有更好的表现，因为剧情根本不需要表演的层次感。我特别希望有机会去现场看谢君豪的话剧，可惜这么好的演员都没大红过，这里也不过是个脸谱化的角色。最好的那批香港演员已经老去，后继有人吗？（关于结尾，再啰嗦两句，既然善恶只在一念间，刘青云离开是因为他相信刘德华会完成最后的爆破，而作为创伤后遗症患者，刘德华其实是可能有迟疑或是选择不爆破的，当然，这不符合编剧一直简单粗暴的人物及情节刻画。）
, user_name:just, comment_time:2020-12-27 22:18:50
comment:
没有滥用特效，节奏得当，老港片的味道，剧本的优缺点都很明显——叙事流畅设计精巧，把原本并不简单容易讲乱的故事线表述得非常清晰，但是人物的心理曲线和行为动机都差一口气，
, user_name:潋 | Béatrice, comment_time:2020-12-27 21:54:43
comment:
1220点映感想：华仔真的不老男神哈哈，就是看电影的时候替他和

In [125]:
post_data = {
    'name':'18664678368',
    'password':'LJC970412',
    'remember':'false'
}

In [127]:
crawl.add_header({
    "Referer":'https://accounts.douban.com/passport/login'
})


In [128]:
user = crawl.session_post('https://accounts.douban.com/j/mobile/login/basic',data=post_data)

In [129]:
user.text

'{"status":"success","message":"success","description":"处理成功","payload":{"account_info":{"name":"Ljc970412","weixin_binded":false,"phone":"18664678368","avatar":{"medium":"https://img3.doubanio.com\\/icon\\/user_large.jpg","median":"https://img1.doubanio.com\\/icon\\/user_normal.jpg","large":"https://img3.doubanio.com\\/icon\\/user_large.jpg","raw":"https://img3.doubanio.com\\/icon\\/user_large.jpg","small":"https://img1.doubanio.com\\/icon\\/user_normal.jpg","icon":"https://img3.doubanio.com\\/pics\\/icon\\/user_icon.jpg"},"id":"217184556","uid":"217184556"}}}'

In [111]:
params = {
            'q':'紧急救援',
            'cat':''
        }

In [140]:
user.text

'{"status":"success","message":"success","description":"处理成功","payload":{"account_info":{"name":"Ljc970412","weixin_binded":false,"phone":"18664678368","avatar":{"medium":"https://img3.doubanio.com\\/icon\\/user_large.jpg","median":"https://img1.doubanio.com\\/icon\\/user_normal.jpg","large":"https://img3.doubanio.com\\/icon\\/user_large.jpg","raw":"https://img3.doubanio.com\\/icon\\/user_large.jpg","small":"https://img1.doubanio.com\\/icon\\/user_normal.jpg","icon":"https://img3.doubanio.com\\/pics\\/icon\\/user_icon.jpg"},"id":"217184556","uid":"217184556"}}}'

In [133]:
res = dict()

In [134]:
from bs4 import BeautifulSoup

In [137]:
soup = BeautifulSoup(response.content, 'html.parser')
results = soup.find(class_='result-list').find_all(class_='result')
for result in results:
    pic = result.find(class_='pic')
    content = result.find(class_='content')
    img = pic.img.get('src')
    link = content.a.get('href')
    name = content.a.text
    description = content.p.text if content.p else ''
    res[name]={
        'img':img,
        'link':link,
        'description':description
    }